# Cuery usage example

In [1]:
import importlib
from pathlib import Path

import instructor
import openai
import pandas as pd

from cuery import pprint, response, utils
from cuery.work import DirceJobs, DirceTasks

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Jobs

In [2]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.csv"
data = pd.read_csv(DATA_DIR)
data.columns = [c.replace(" ", "_") for c in data.columns]
display(data)

n_examples = 5
context_df = data.iloc[:5]
context_dict = {col: context_df[col].tolist() for col in context_df.columns}
context_records = context_df.to_dict(orient="records")

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4
1,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2
2,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5
3,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5
4,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9
5,46 Comercio al por mayor e intermediarios del ...,464 Comercio al por mayor de artículos de uso ...,228324,-2.0
6,"47 Comercio al por menor, excepto de vehículos...",471 Comercio al por menor en establecimientos ...,233708,1.3
7,"47 Comercio al por menor, excepto de vehículos...",477 Comercio al por menor de otros artículos e...,242341,-1.3
8,49 Transporte terrestre y por tubería,493 Otro transporte terrestre de pasajeros,142413,0.5
9,49 Transporte terrestre y por tubería,494 Transporte de mercancías por carretera y s...,273002,-1.5


In [3]:
from cuery.work import tasks

task = tasks.DirceJobs
pprint(task)

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── Prompt ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │  Required: ['Division', 'Actividad_principal']                                                              │ │
│ │                                                                                                             │ │
│ │ ╭──────────────────────────────────────────────── SYSTEM ─────────────────────────────────────────────────╮ │ │
│ │ │                                                                                                         │ │ │
│ │ │  You're an analyst at the Spanish 'Instituo Nacional de Estadística' (INE) analyzing data from its      │ │ │
│ │ │  'Directorio Central de Empresas' (DIRCE). Your objective is to analyze groups of companies, identifie  │ │ │
│ │ │  by a sector ('Division') and a corresponding main activity ('Actividad principal') in order to identi  │ │ │
│ │ │  jobs within those companies that are likely to be automatable by AI. Both 'Division' and               │ │ │
│ │ │  'Actividad_principal' are provided in Spanish and may include numeric IDs that you can ignore if you   │ │ │
│ │ │  don't understand them. Always respond in English. Only consider jobs that are computer- or paper-base  │ │ │
│ │ │  and can be automated by AI using software (don't include jobs automatable by robots or other physical  │ │ │
│ │ │  means).                                                                                                │ │ │
│ │ │                                                                                                         │ │ │
│ │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │ │
│ │ ╭───────────────────────────────────────────────── USER ──────────────────────────────────────────────────╮ │ │
│ │ │                                                                                                         │ │ │
│ │ │  Please analyze the following jobs sector and identify jobs that are automatable by AI software.        │ │ │
│ │ │  Division: {{Division}}                                                                                 │ │ │
│ │ │  Actividad principal: {{Actividad_principal}}                                                           │ │ │
│ │ │                                                                                                         │ │ │
│ │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│ ╭────────────────────────────────────────────── RESPONSE: Jobs ───────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │ ╭─ jobs: list[cuery.work.tasks.Job] ──────────────────────────────────────────────────────────────────────╮ │ │
│ │ │                                                                                                         │ │ │
│ │ │  A list of jobs with their AI automation potential and reasons for that potential                       │ │ │
│ │ │                                                                                                         │ │ │
│ │ │  {'required': True, 'metadata': [MinLen(min_length=3)]}                                                 │ │ │
│ │ │                                                   

In [4]:
jobs = await task(context_df, model="gpt-4.1-mini", n_concurrent=10)
jobs

Gathering responses: 100%|██████████| 5/5 [00:04<00:00,  1.18it/s]


[Jobs(jobs=[Job(name='Meat Processing Data Clerk', description='Manages and records data related to meat processing activities, including tracking batches and quality control records.', automation_potential=8, reason='Routine data entry and record-keeping can be automated by AI software.'), Job(name='Product Compliance Analyst', description='Monitors compliance with safety and quality standards for meat products through document analysis.', automation_potential=7, reason='Document review and compliance flagging can be aided by AI tools.'), Job(name='Inventory Control Coordinator', description='Tracks inventory levels, orders, and supplies for the meat processing plant using software systems.', automation_potential=9, reason='Inventory tracking and order management are highly automatable by AI software.')]),
 Jobs(jobs=[Job(name='Administrative Clerk', description='Handles documentation, data entry, and record keeping within vehicle parts manufacturing.', automation_potential=8, reason=

In [5]:
jobs_df = DirceJobs.explode_responses(jobs, context_df)
jobs_df = jobs_df.rename(
    columns={
        "name": "job",
        "description": "job_description",
        "automation_potential": "job_automation_potential",
        "reason": "job_automation_reason",
    }
)
jobs_df

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,job,job_description,job_automation_potential,job_automation_reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Meat Processing Data Clerk,Manages and records data related to meat proce...,8,Routine data entry and record-keeping can be a...
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Product Compliance Analyst,Monitors compliance with safety and quality st...,7,Document review and compliance flagging can be...
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Inventory Control Coordinator,"Tracks inventory levels, orders, and supplies ...",9,Inventory tracking and order management are hi...
3,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Administrative Clerk,"Handles documentation, data entry, and record ...",8,Routine data entry and document processing tas...
4,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Inventory Manager,Manages stock levels and orders for components...,7,AI can optimize inventory tracking and reorder...
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Quality Control Analyst,Evaluates product quality using reports and da...,6,AI can assist in analyzing quality data but ph...
6,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Electrical Installer,Installs and maintains electrical wiring and s...,6,Electrical system design and diagnostics can b...
7,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Plumbing Installer,Installs and repairs plumbing systems includin...,4,"AI can assist in planning and diagnostics, but..."
8,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Construction Project Administrator,"Manages documentation, scheduling, and communi...",9,"AI software can automate scheduling, documenta..."
9,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Safety Compliance Officer,Ensures construction activities comply with sa...,7,AI can monitor compliance data and generate re...


In [10]:
usage = response.token_usage(jobs)
print(usage)
usage.sum()

   prompt  completion
0     433         139
1     443         120
2     436         136
3     430         123
4     446         132


prompt        2188
completion     650
dtype: int64

In [17]:
# jobs with Perplexity (using its OpenAI-compatible API)
import os

utils.set_api_keys()

perplex = openai.AsyncOpenAI(
    api_key=os.getenv("PERPLEXITY_API_KEY"), base_url="https://api.perplexity.ai"
)
perplex = instructor.from_perplexity(perplex)

jobs_perp = await DirceJobs(context_df, client=perplex, model="sonar", n_concurrent=2)

Gathering responses: 100%|██████████| 5/5 [06:37<00:00, 79.58s/it] 

Encountered: 5 response parsing errors!


In [19]:
DirceJobs.explode_responses(jobs_perp, context_df)

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,name,description,automation_potential,reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Inventory Management Clerk,"Tracks and manages stock levels, orders, and d...",9,"AI can automate stock tracking, ordering, and ..."
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Assurance Data Analyst,Analyzes quality control data from production ...,9,"AI excels at processing large datasets, detect..."
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Order Processing & Customer Service Rep. (Digi...,"Handles order entry, customer inquiries via em...",8,AI-driven systems can process orders and respo...
3,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Regulatory Compliance Documentation Specialist,Prepares and maintains compliance documentatio...,7,AI tools can integrate regulatory updates and ...
4,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Production Planning and Scheduling Officer,Schedules production runs based on demand fore...,9,AI systems can analyze demand patterns and res...
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Design & CAD Technician,AI can automate vehicle component design tasks...,8,AI-driven design and simulation tools automate...
6,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Quality Control Inspector (Software-Based),AI detects defects in vehicle parts via softwa...,9,AI-based systems fully automate defect detecti...
7,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Supply Chain Data Analyst,AI forecasts demand and manages inventory auto...,8,AI software can handle predictive modeling and...
8,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Procurement Clerk,AI can automate purchase orders and vendor man...,7,"Routine, data-driven purchasing tasks are auto..."
9,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Production Planner,AI optimizes manufacturing schedules using rea...,8,AI excels at scheduling and planning using sof...


## Tasks

In [7]:
tasks = await DirceTasks(jobs_df, model="gpt-4.1-mini", n_concurrent=20)
tasks

Gathering responses: 100%|██████████| 15/15 [00:13<00:00,  1.08it/s]

Encountered: 1 response parsing errors!


[JobTasks(tasks=[JobTask(name='Data Entry of Quality Measurements', description='Automate the input of quality control data such as temperature, pH levels, and microbial counts into databases.', automation_potential=9, intelligence=3, sexyness=4, scalability=8, data_needs='Structured numeric and text data from lab instruments and inspection reports.', products=['Tableau', 'Microsoft Power BI', 'Zoho Analytics']), JobTask(name='Visual Inspection Analysis', description='Use AI image recognition to identify defects or inconsistencies in product appearance during visual inspection.', automation_potential=8, intelligence=6, sexyness=7, scalability=7, data_needs='High-resolution images or video of products under inspection.', products=['Google Cloud Vision', 'Amazon Rekognition', 'Clarifai']), JobTask(name='Report Generation', description='Automatically generate quality control reports summarizing inspection data and compliance status.', automation_potential=9, intelligence=5, sexyness=6, sc

In [6]:
tasks_df = DirceTasks.explode_responses(tasks, jobs_df)
tasks_df = tasks_df.rename(columns={"name": "task", "description": "task_description"})
tasks_df

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,job,job_description,job_automation_potential,job_automation_reason,task,task_description,automation_potential,intelligence,sexyness,scalability,data_needs,products
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Data Entry and Report Generation,Automate inputting quality control data and ge...,8,4,5,8,Requires quality control measurements and hist...,"[Tableau, Power BI, Zoho Analytics]"
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Defect Detection in Product Images,Use AI to analyze product images to identify d...,9,7,6,7,Needs access to labeled product images with de...,"[Google Vision AI, Amazon Rekognition, IBM Wat..."
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Quality Parameter Monitoring,Automatically monitor and analyze quality para...,8,6,4,7,Requires real-time sensor data of processing e...,"[OSIsoft PI System, Honeywell Process Historia..."
3,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Responsible for reviewing and documenting qual...,8,Routine data analysis and pattern recognition ...,Compliance Documentation Automation,Generate compliance documentation and certific...,7,5,3,6,Quality control data and relevant regulatory s...,"[DocuSign, Adobe Sign, Formstack]"
4,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Inventory Data Coordinator,Manages inventory records and updates stock da...,9,High-volume data entry and database updates ar...,Automated Data Entry,Automatically input inventory data into comput...,9,4,5,9,Requires structured inventory data and input f...,"[UiPath, Automation Anywhere, ABBYY FlexiCapture]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Order Processing,"Automate order entry, validation, and status u...",8,4,5,8,Sales orders and customer data for processing ...,"[Salesforce Einstein, SAP Conversational AI]"
68,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Customer Inquiry Response,AI chatbots to handle common customer question...,9,6,7,9,Frequently asked questions and customer intera...,"[Zendesk Answer Bot, IBM Watson Assistant]"
69,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Complaint Logging and Tracking,Automatically log and track customer complaint...,7,5,4,7,Complaint data and resolution status information.,"[Freshdesk, Zoho Desk]"
70,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Responds to customer inquiries and resolves is...,7,Routine queries and responses can be automated...,Inventory Status Updates,Automate communication of product availability...,8,4,6,8,Real-time inventory data and product catalogs.,"[TradeGeck

In [ ]:
tasks_df.to_csv(
    GDRIVE / "Research/future_of_work/outputs/ine_dirce_ai_tasks.csv",
    index=False,
)